## Examples for loading data relevant to decoding, demonstration of some helper functions

### Imports / Setup

In [14]:
import os
import numpy as np
import pandas as pd

Here we'll specify the U19_NAS path as `/data/`, since that's how it's mounted on the docker container, but change this variable however

In [15]:
U19_NAS_base = "/data/"

### See what a sessions dataframe looks like
- Only requirement here is that this dataframe has a session_name column, that identifies each session

In [16]:
# path is patrick_scratch because this is a derived list of sessions used for decoding, selected by certain conditions
SESSIONS_PATH = os.path.join(U19_NAS_base, "patrick_scratch/multi_sess/valid_sessions.pickle")
valid_sess = pd.read_pickle(SESSIONS_PATH)
valid_sess[:10]

,session_datetime,session_count,session_name,valid
5,2018-07-09,0,20180709,True
6,2018-09-18,0,20180918,True
7,2018-09-12,0,20180912,True
10,2018-10-08,0,20181008,True
12,2018-10-02,0,20181002,True
13,2018-08-06,0,20180806,True
14,2018-06-25,0,20180625,True
16,2018-06-22,0,20180622,True
19,2018-09-06,0,20180906,True
20,2018-08-01,0,20180801,True


### Just look at one example session for now

In [20]:
sess_name = "20180802"

### Load behavior for example session

In [17]:
# behavioral data for specific session stored as a csv:
sess_behavior_path = os.path.join(U19_NAS_base, f"rawdata/sub-SA/sess-{sess_name}/behavior/sub-SA_sess-{sess_name}_object_features.csv")
behavior = pd.read_csv(sess_behavior_path)
behavior

,TrialNumber,BlockNumber,TrialAfterRuleChange,TaskInterrupt,ConditionNumber,Response,ItemChosen,TrialType,CurrentRule,LastRule,...,Item0Pattern,Item1Shape,Item1Color,Item1Pattern,Item2Shape,Item2Color,Item2Pattern,Item3Shape,Item3Color,Item3Pattern
0,0,0,0,NaN,5276,Correct,0.0,1,RIPPLE,NaN,...,RIPPLE,STAR,GREEN,POLKADOT,CIRCLE,CYAN,ESCHER,SQUARE,MAGENTA,SWIRL
1,1,0,1,NaN,5628,Correct,0.0,1,RIPPLE,NaN,...,RIPPLE,STAR,MAGENTA,POLKADOT,TRIANGLE,YELLOW,ESCHER,CIRCLE,GREEN,SWIRL
2,2,0,2,NaN,5752,Correct,0.0,1,RIPPLE,NaN,...,RIPPLE,TRIANGLE,MAGENTA,POLKADOT,STAR,CYAN,ESCHER,CIRCLE,YELLOW,SWIRL
3,3,0,3,NaN,5691,Correct,0.0,1,RIPPLE,NaN,...,RIPPLE,STAR,MAGENTA,POLKADOT,CIRCLE,GREEN,ESCHER,TRIANGLE,CYAN,SWIRL
4,4,0,4,NaN,5523,Correct,0.0,1,RIPPLE,NaN,...,RIPPLE,CIRCLE,YELLOW,POLKADOT,SQUARE,GREEN,ESCHER,TRIANGLE,CYAN,SWIRL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757,1757,55,14,NaN,3873,NoFixation,NaN,6,YELLOW,GREEN,...,SWIRL,CIRCLE,CYAN,RIPPLE,TRIANGLE,MAGENTA,ESCHER,SQUARE,GREEN,POLKADOT
1758,1758,55,15,NaN,3806,Late,NaN,6,YELLOW,GREEN,...,RIPPLE,TRIANGLE,CYAN,SWIRL,SQUARE,MAGENTA,POLKADOT,CIRCLE,GREEN,ESCHER
1759,1759,55,16,NaN,3515,NoFixation,NaN,6,YELLOW,GREEN,...,RIPPLE,STAR,CYAN,POLKADOT,CIRCLE,MAGENTA,SWIRL,SQUARE,GREEN,ESCHER
1760,1760,55,17,NaN,3472,NoFixation,NaN,6,YELLOW,GREEN,...,POLKADOT,STAR,CYAN,ESCHER,SQUARE,MAGENTA,RIPPLE,CIRCLE,GREEN,SWIRL


In [18]:
behavior.columns

Index(['TrialNumber', 'BlockNumber', 'TrialAfterRuleChange', 'TaskInterrupt',
       'ConditionNumber', 'Response', 'ItemChosen', 'TrialType', 'CurrentRule',
       'LastRule', 'ShiftType', 'Perseveration', 'TrialStart', 'FixCrossOn',
       'FixationOnCross', 'FixRequireEnd', 'StimOnset', 'FeedbackEncode',
       'FeedbackOnset', 'TrialEnd', 'Item0_xPos', 'Item0_yPos', 'Item1_xPos',
       'Item1_yPos', 'Item2_xPos', 'Item2_yPos', 'Item3_xPos', 'Item3_yPos',
       'Item0Shape', 'Item0Color', 'Item0Pattern', 'Item1Shape', 'Item1Color',
       'Item1Pattern', 'Item2Shape', 'Item2Color', 'Item2Pattern',
       'Item3Shape', 'Item3Color', 'Item3Pattern'],
      dtype='object')

Note the important columns here:
- TrialNumber: unique identifier for trial
- BlockNumber: unique identifier for block
- ItemChosen: which card index is chosen, 0-3
- Reponse: Correct/Incorrect, can also be NoFixation/Late etc. 
- CurrentRule: Rule of block
- Item{0/1/2/3}{Color/Shape/Pattern}: color/shape/pattern of each card
- FeedbackOnset: time of feedbackonset in ms, using this for alignment

There are also some useful helper functions, that are used in the `decode_features_with_pseudo.py` script

In [28]:
import utils.behavioral_utils as behavioral_utils

# filter trials
valid_beh = behavior[behavior.Response.isin(["Correct", "Incorrect"])]
# per trial finds the color/shape/pattern of the selected card
behavioral_utils.get_selection_features(valid_beh)

,TrialNumber,Color,Shape,Pattern
0,0,YELLOW,TRIANGLE,RIPPLE
1,1,CYAN,SQUARE,RIPPLE
2,2,GREEN,SQUARE,RIPPLE
3,3,YELLOW,SQUARE,RIPPLE
4,4,MAGENTA,STAR,RIPPLE
...,...,...,...,...
1744,1745,YELLOW,SQUARE,POLKADOT
1745,1746,MAGENTA,SQUARE,POLKADOT
1746,1747,YELLOW,CIRCLE,SWIRL
1747,1748,CYAN,CIRCLE,SWIRL


### Load example spikes (pre-binned, pre-aligned)

In [22]:
# pre-aligned and binned spikes are all relative to some trial event and interval before/after, so define those. 
pre_interval = 1300
event = "FeedbackOnset"
post_interval = 1500
interval_size = 100

In [25]:
sess_spikes_path = os.path.join(U19_NAS_base, f"patrick_scratch/multi_sess/{sess_name}/{sess_name}_firing_rates_{pre_interval}_{event}_{post_interval}_{interval_size}_bins.pickle")
spikes = pd.read_pickle(sess_spikes_path)
spikes

,TrialNumber,UnitID,TimeBins,SpikeCounts,FiringRate
0,0.0,0,0.0,0.0,0.0
1,0.0,0,0.1,0.0,0.0
2,0.0,0,0.2,0.0,0.0
3,0.0,0,0.3,0.0,0.0
4,0.0,0,0.4,0.0,0.0
...,...,...,...,...,...
2252707,1749.0,45,2.3,0.0,0.0
2252708,1749.0,45,2.4,0.0,0.0
2252709,1749.0,45,2.5,0.0,0.0
2252710,1749.0,45,2.6,0.0,0.0


Note the columns of use: 
- TrialNumber: unique identifier of trial, needs to match with behavioral trial number
- UnitID: unique identifier of units in session
- TimeBins: this is in seconds, starts at 0.0. 
- SpikeCounts: this is the column traditionally used for decoding